In [1]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180


import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os


Autosaving every 180 seconds


C:\Users\mnzp0\AppData\Local\Temp\ipykernel_17332\4197309928.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


############### PREDICT NEXT NEURAL TIME SERIES POINT ################ 

Here we are just trying to get the transformer to learn the dynamics of raw (i.e. PCA denoised) neural time series.

#### Step 1: One brain area

- input shape [time_points] = [40000]    # this is just a single time series from the visualizion notebook
- label: [time_points[1:]]                # here we predict the time series but shifted by 1

This is exactly what transformers are developed to do, so we shouldn't have to do too much work to adapt them. We can also smooth or bin the neural data as it's abit noisy. 

##### Major challenges:

1. Figure out how to feed continous time series into the transformer.

There are some methods already out there

https://huggingface.co/blog/time-series-transformers

https://huggingface.co/docs/transformers/model_doc/time_series_transformer


#### Step 2: Multiple brain areas

- input shape [time_points, n_areas] = [40000, 30]     #  
- label: [time_points[1:], 30]                         # 

##### Major challenges:

1. So we would need to extend the above to work with multiple cortical areas...


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import glob
from sklearn.preprocessing import StandardScaler

In [3]:
#Loading The Data
root_dir = 'data'
mouse_id = 'IA1'
session_id = 'Feb_16'

# load the raw data from each neural area
animal_dir = os.path.join(root_dir, mouse_id, session_id)

# find the file using glob that has "wholestack.npz" in it
fname = glob.glob(os.path.join(animal_dir, '*wholestack.npz'))[0]

data_stack = np.load(fname, allow_pickle=True)
data = data_stack['whole_stack']

In [4]:
# Slice - What amount to look at to reduce the computational complexity for the 40,000 - Increase Later
slice = 1000

if slice == 0:  # Default to the length of the data if a slice is not being used
    slice = len(torch.tensor(data[1]))


# Starting off lets just use the start of the tensor

# Convert to PyTorch tensors
neural_data = torch.tensor(data, dtype=torch.float32)[:slice]
seq_len = slice

time_steps = torch.arange(seq_len, dtype=int)  # Creates a tensor [0, 1, 2, ..., seq_len-1]

# Doing this incorrect, need to seperate the different areas not just the two area gaps

# Create dataset and dataloader
dataset = TensorDataset(neural_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# Need to create Test, Validation, and Train Sets
def preprocess_data(data, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_state=None):
    """
    Preprocess neural data by splitting into train/val/test sets and normalizing.
    
    Args:
        data: Input array of shape [spatial_dims, time_points]
        train_ratio: Proportion of data for training (default: 0.7)
        val_ratio: Proportion of data for validation (default: 0.15)
        test_ratio: Proportion of data for testing (default: 0.15)
        random_state: Random seed for reproducibility
        
    Returns:
        dict: Contains train, val, and test splits (both normalized and raw)
    """
    # Verify ratios sum to 1
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-10, "Ratios must sum to 1"
    
    # Get dimensions
    n_spatial, n_time = data.shape
    
    # Create indices and optionally shuffle
    indices = np.arange(n_time)
    if random_state is not None:
        rng = np.random.RandomState(random_state)
        rng.shuffle(indices)
    
    # Calculate split points
    train_end = int(n_time * train_ratio)
    val_end = int(n_time * (train_ratio + val_ratio))
    
    # Split the data
    train_data = data[:, indices[:train_end]]
    val_data = data[:, indices[train_end:val_end]]
    test_data = data[:, indices[val_end:]]
    
    # Initialize and fit scaler on training data
    scaler = StandardScaler()
    train_2d = train_data.transpose()
    scaler.fit(train_2d)
    
    # Transform all splits
    train_norm = scaler.transform(train_2d).transpose()
    val_norm = scaler.transform(val_data.transpose()).transpose()
    test_norm = scaler.transform(test_data.transpose()).transpose()
    
    return {
        'train': train_norm,
        'val': val_norm,
        'test': test_norm,
        'train_raw': train_data,
        'val_raw': val_data,
        'test_raw': test_data,
        'scaler': scaler
    }


